In [1]:
import pyspark
from pyspark.sql.functions import *

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("advance").getOrCreate()

23/01/24 21:40:45 WARN Utils: Your hostname, codeStation resolves to a loopback address: 127.0.1.1; using 192.168.88.83 instead (on interface wlo1)
23/01/24 21:40:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/24 21:40:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/24 21:40:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
%%sh
ls

basics_review_pyspark.ipynb
going_advanced_10_15.ipynb
nyc-collisions-2019-reduced.csv


In [5]:
# Lets check the current spark conf for AQE and shuffle partitions
print(spark.conf.get("spark.sql.adaptive.enabled"))
print(spark.conf.get("spark.sql.adaptive.coalescePartitions.enabled"))
print(spark.conf.get("spark.sql.shuffle.partitions"))
print(spark.conf.get("spark.sql.adaptive.advisoryPartitionSizeInBytes")) #approx 64MB Default

true
true
200
67108864b


In [ ]:
spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False)
spark.conf.set("spark.sql.shuffle.partitions", 180)

In [10]:
# Read example data set
df = spark.read.format("csv").option("header", True).load("nyc-collisions-2019-reduced.csv")
df.printSchema()
print("Initial Partition after read: " + str(df.rdd.getNumPartitions()))

root
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- injured: string (nullable = true)
 |-- killed: string (nullable = true)
 |-- cause: string (nullable = true)

Initial Partition after read: 3


In [11]:
# GroupBy opeartion to trigger Shuffle
# Since our output with city_id as group by is smaller than < 64MB thus the data is written in single partiton
from pyspark.sql.functions import sum
df_count = df.select("borough","injured").groupBy("borough").agg(sum("injured"))
print("Output shuffle partitions: " + str(df_count.rdd.getNumPartitions()))

Output shuffle partitions: 1


In [15]:
# GroupBy opeartion to trigger Shuffle but this time with zipcode(which is more unique - thus more data)
# Since our output with zipcode as group by is > 64MB thus the data is written in multiple partitions
from pyspark.sql.functions import sum

df_count = df.selectExpr("date","killed").groupBy("date").agg(sum("killed"))

print("Output shuffle partitions: " + str(df_count.rdd.getNumPartitions()))

Output shuffle partitions: 1


In [ ]:
df_zipped = spark \
    .read \
    .format("csv") \
    .option("compression", "gzip") \
    .option("header", True) \
    .load("dataset/tmp/sales.csv.gz")

In [16]:
df_spark_png = spark \
    .read \
    .format("binaryFile") \
    .load("vizheads.csv")

df_spark_png.printSchema()
df_spark_png.show()

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)



+--------------------+--------------------+------+--------------------+
|                path|    modificationTime|length|             content|
+--------------------+--------------------+------+--------------------+
|file:/run/media/s...|2023-01-24 21:54:...| 99936|[63 75 73 74 6F 6...|
+--------------------+--------------------+------+--------------------+



In [20]:
# Lets generate the text file back from the binary content
byte_content = df_spark_png.select("content").collect()[0][0]

# Lets write the byte content as file back
with open("new_example.txt", "w") as f:
    f.write(str(byte_content))
    f.close()

In [24]:
# gaining performance benefits
# Set up to read from JDBC SQLite database
driver: str = "org.sqlite.JDBC"
db_path: str = "dataset/jdbc/demo-sqlite.db"
jdbc_url: str = "jdbc:sqlite:" + db_path
table_name: str = "sales_csv"

In [ ]:
# Lets read the SQLite table using JDBC driver and validate the data
df = spark \
    .read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", jdbc_url) \
    .option("dbtable", table_name) \
    .load()

In [ ]:
# Checking the explain plan
df.explain(True)

In [ ]:
def x():
    df_full = spark \
    .read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", jdbc_url) \
    .option("dbtable", table_name) \
    .load()

In [ ]:
df_filtered = spark \
    .read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", jdbc_url) \
    .option("dbtable", table_name) \
    .load() \
    .filter("city_id = 216510442")
    

In [ ]:
 df_filtered = spark \
    .read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", jdbc_url) \
    .option("dbtable", table_name) \
    .load() \
    .cache() \
    .filter("city_id = 216510442")

In [ ]:
pushDownQuery = """(select city_id, count(1) as cnt from sales_csv group by city_id) as sales_csv"""
@get_time
def x():
    df_filtered = spark \
    .read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", jdbc_url) \
    .option("dbtable", pushDownQuery) \
    .load()

In [ ]:
df_full = spark \
    .read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", jdbc_url) \
    .option("dbtable", table_name) \
    .load()

df_full.selectExpr("min(trx_id) as min_trx_id", "max(trx_id) as max_trx_id").show()

In [ ]:
df_full = spark \
    .read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", jdbc_url) \
    .option("dbtable", table_name) \
    .option("partitionColumn", "trx_id") \
    .option("lowerBound", 20) \
    .option("upperBound", 2147474653) \
    .option("numPartitions", 8) \
    .load()
    
    df_full.write.format("noop").mode("overwrite").save()
    df_full.printSchema()
    print("Number of Partitons: "+ str(df_full.rdd.getNumPartitions()))

In [ ]:
df_full = spark \
    .read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", jdbc_url) \
    .option("dbtable", table_name) \
    .option("partitionColumn", "trx_id") \
    .option("lowerBound", 20) \
    .option("upperBound", 2147474653) \
    .option("numPartitions", 8) \
    .option("fetchsize", 8000) \
    .load()
    
    df_full.write.format("noop").mode("overwrite").save()
    df_full.printSchema()
    print("Number of Partitons: "+ str(df_full.rdd.getNumPartitions()))

In [ ]:
### Count(1) vs Count(*) vs Count(col_name)

df.groupBy("trx_id").agg(count("*")).explain(True)

df.groupBy("trx_id").agg(count(lit(1))).explain(True)

df.groupBy("trx_id").agg(count("city_id")).explain(True)

In [25]:
# Broadcast Varible

dept_names = {"D001": "Department 1", "D002": "Department 2", "D003": "Department 3"}
dept_est = {"D001": 1990, "D003": 2001}

broadcast_dept_names = spark.sparkContext.broadcast(dept_names)
broadcast_dept_est = spark.sparkContext.broadcast(dept_est)

# Check the type of the variable
print("The type is: " + str(type(broadcast_dept_est)))

# In case we want to check the value
print(broadcast_dept_names.value)
print(broadcast_dept_est.value)

The type is: <class 'pyspark.broadcast.Broadcast'>
{'D001': 'Department 1', 'D002': 'Department 2', 'D003': 'Department 3'}
{'D001': 1990, 'D003': 2001}


In [ ]:
#Lets use broadcast variable to use Dept info
_new_schema = ["NAME", "DEPT_CODE", "FAV_SUBJECT", "DEPT_NAME", "ESTD"]

# Use lambda function to iterate over row to get the broadcast value
df = df_students.rdd.map(lambda row: [
    row.NAME, 
    row.DEPT_CODE, 
    row.FAV_SUBJECT,
    broadcast_dept_names.value.get(row.DEPT_CODE),
    broadcast_dept_est.value.get(row.DEPT_CODE)]).toDF(_new_schema)

df.show()

In [ ]:
#Reading partitioned files

# Read with Schema
_schema = "transacted_at timestamp, trx_id int, retailer_id int, description string, amount decimal(38,18), city_id int, trx_year int, trx_month int, trx_date int"
df_2 = spark.read \
  .format("parquet") \
  .schema(_schema) \
  .load("/user/hive/delta-warehouse/sales_partitioned.parquet")

In [ ]:
# Register the data as table
_create_sql = """
CREATE TABLE default.sales_partitioned USING PARQUET LOCATION '/user/hive/delta-warehouse/sales_partitioned.parquet'
"""
_refresh_sql = """
MSCK repair table default.sales_partitioned
"""
# Run the SQL commands
spark.sql(_create_sql)
spark.sql(_refresh_sql)

In [ ]:
#This command is read from the catalog
df_3 = spark.read.table("default.sales_partitioned")

In [ ]:
# Determine the degree of parallelism
spark.sparkContext.defaultParallelism

In [ ]:
# Disable all AQE optimization for benchmarking tests
spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.adaptive.localShuffleReader.enabled", False)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False)

In [ ]:
# Determine the degree of parallelism
spark.sparkContext.defaultParallelism

In [ ]:
# Define shuffle partitions which is not Factor of core
spark.conf.set("spark.sql.shuffle.partitions", 9)

In [ ]:
# Not repartitiong with factor
spark.read.format("parquet").load("dataset/sales.parquet/").repartition(9).write.format("noop").mode("overwrite").save()

In [ ]:
 Repartitiong based on factor of cores
spark.read.format("parquet").load("dataset/sales.parquet/").repartition(8).write.format("noop").mode("overwrite").save()

In [ ]:

def fix_header(df: DataFrame) -> list:
    fixed_col_list: list = []
    for col in df.columns:
        fixed_col_list.append(f"`{str(col).strip()}` as {str(col).strip().replace(' ','_').lower()}")
        
    return fixed_col_list

In [26]:
# Validate if catalog is Hive
spark.conf.get("spark.sql.catalogImplementation")

'in-memory'

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Metastore Without HiveSupport") \
    .master("local[*]") \
    .config("spark.sql.warehouse.dir", "spark-warehouse") \
    .getOrCreate()


## !pip install sparksql-magic

In [29]:
%load_ext sparksql_magic